credit to:
 - https://www.kaggle.com/knowledgegrappler/embeddings-features-tdf-idf-let-s-party/comments

In [2]:
import numpy as np
np.random.seed(1)
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

#CREATE TARGET VARIABLE
train["EAP"] = (train.author=="EAP")*1
train["HPL"] = (train.author=="HPL")*1
train["MWS"] = (train.author=="MWS")*1
train.drop("author", 1, inplace=True)
target_vars = ["EAP", "HPL", "MWS"]
train.head(2)

,id,text,EAP,HPL,MWS
0,id26305,"This process, however, afforded me no means of...",1,0,0
1,id17569,It never once occurred to me that the fumbling...,0,1,0


In [3]:
from nltk.corpus import stopwords
eng_stopwords = set(stopwords.words("english"))
import string
## Number of words in the text ##
train["num_words"] = train["text"].apply(lambda x: len(str(x).split()))
test["num_words"] = test["text"].apply(lambda x: len(str(x).split()))

## Number of unique words in the text ##
train["num_unique_words"] = train["text"].apply(lambda x: len(set(str(x).split())))
test["num_unique_words"] = test["text"].apply(lambda x: len(set(str(x).split())))

## Number of stopwords in the text ##
train["num_stopwords"] = train["text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
test["num_stopwords"] = test["text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))

## Number of punctuations in the text ##
train["num_punctuations"] =train['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )
test["num_punctuations"] =test['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )

In [32]:
def punct_pct2(text, punc):
    count = sum([1 for char in text if char==punc])
    return round(count/(len(text) - text.count(" ")), 3)*100

In [34]:
train["comma%"] = train.text.apply(lambda x: punct_pct2(x, ','))
train["semicolon%"] = train.text.apply(lambda x: punct_pct2(x, ';'))
train["colon%"] = train.text.apply(lambda x: punct_pct2(x, ':'))
train["question%"] = train.text.apply(lambda x: punct_pct2(x, '?'))

test["comma%"] = test.text.apply(lambda x: punct_pct2(x, ','))
test["semicolon%"] = test.text.apply(lambda x: punct_pct2(x, ';'))
test["colon%"] = test.text.apply(lambda x: punct_pct2(x, ':'))
test["question%"] = test.text.apply(lambda x: punct_pct2(x, '?'))
train.head()

,id,text,EAP,HPL,MWS,num_words,num_unique_words,num_stopwords,num_punctuations,num_words_upper,num_words_title,mean_word_len,stem_text,seq_text_stem,comma%,semicolon%,colon%,question%
0,id26305,"This process, however, afforded me no means of...",1,0,0,41,35,19,7,2,3,4.658537,this process howev afford me no mean of asce...,"[27, 1876, 161, 743, 22, 37, 201, 2, 1652, 1, ...",2.1,1.0,0.0,0.0
1,id17569,It never once occurred to me that the fumbling...,0,1,0,14,14,8,1,0,1,4.142857,it never onc occur to me that the fumbl might ...,"[10, 99, 138, 672, 4, 22, 9, 1, 3675, 85, 23, ...",0.0,0.0,0.0,0.0
2,id11008,"In his left hand was a gold snuff box, from wh...",1,0,0,36,32,16,5,0,1,4.583333,in his left hand was a gold snuff box from wh...,"[7, 15, 164, 122, 8, 6, 935, 4166, 636, 24, 18...",2.4,0.0,0.0,0.0
3,id27763,How lovely is spring As we looked from Windsor...,0,0,1,34,32,13,4,0,4,5.088235,how love is spring as we look from windsor ter...,"[133, 106, 26, 749, 16, 35, 94, 24, 903, 2393,...",1.7,0.0,0.0,0.0
4,id12958,"Finding nothing else, not even gold, the Super...",0,1,0,27,25,11,4,0,2,5.481481,find noth els not even gold the superintend ...,"[207, 194, 882, 20, 67, 935, 1, 3204, 1281, 15...",1.4,0.7,0.0,0.0


In [7]:
## Number of upper case words in the text ##
train["num_words_upper"] = train["text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
test["num_words_upper"] = test["text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))

In [8]:
## Number of title words in the text ##
train["num_words_title"] = train["text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
test["num_words_title"] = test["text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))

In [10]:
## Average length of the words in the text ##
train["mean_word_len"] = train["text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
test["mean_word_len"] = test["text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

In [35]:
num_vars = ["mean_word_len", "num_words_title", "num_words_upper", 
            "comma%", "semicolon%", "question%", "colon%",
            "num_punctuations", "num_stopwords", "num_unique_words", "num_words"]

In [45]:
#STEMMING WORDS
import nltk.stem as stm
import re
stemmer = stm.SnowballStemmer("english")
train["stem_text"] = train.text.apply(lambda x: (" ").join([stemmer.stem(z) for z in re.sub("[^a-zA-Z0-9]"," ", x).split(" ")]))
test["stem_text"] = test.text.apply(lambda x: (" ").join([stemmer.stem(z) for z in re.sub("[^a-zA-Z0-9]"," ", x).split(" ")]))

#PROCESS TEXT: RAW
from keras.preprocessing.text import Tokenizer
# tok_raw = Tokenizer()
# tok_raw.fit_on_texts(train.text.str.lower())
tok_stem = Tokenizer()
tok_stem.fit_on_texts(train.stem_text.str.lower())
train["seq_text_stem"] = tok_stem.texts_to_sequences(train.stem_text)
test["seq_text_stem"] = tok_stem.texts_to_sequences(test.stem_text)

In [40]:
#EXTRACT DATA FOR KERAS MODEL
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline

def get_keras_data(dataset, maxlen=20, scaler=None, tdfidf=None):
    if scaler==None:
        scaler = StandardScaler()
        scaler.fit(dataset[num_vars])
    if tdfidf==None:
        tdfidf = Pipeline(steps=[('tdfidf', TfidfVectorizer(analyzer='word', binary=False, ngram_range=(1, 3), stop_words="english")),
                                 ('svd', TruncatedSVD(algorithm='randomized', n_components=20, n_iter=10, random_state=None, tol=0.0) )])
        tdfidf.fit(dataset.text)
    X = {
        "stem_input": pad_sequences(dataset.seq_text_stem, maxlen=maxlen),
        "num_input": scaler.transform(dataset[num_vars]),
        "svd_vect": tdfidf.transform(dataset.text)
    }
    return X, scaler, tdfidf

In [41]:
maxlen = 60
dtrain, dvalid = train_test_split(train, random_state=1, test_size=0.3)
print("processing train...")
X_train, scaler, tdfidf = get_keras_data(dtrain, maxlen)
y_train = np.array(dtrain[target_vars])
print("processing valid...")
X_valid, _, _ = get_keras_data(dvalid, maxlen, scaler, tdfidf)
y_valid = np.array(dvalid[target_vars])
print("processing test...")
X_test, _, _ = get_keras_data(test, maxlen, scaler, tdfidf)

n_stem_seq = np.max( [np.max(X_valid["stem_input"]), np.max(X_train["stem_input"])])+1

processing train...


C:\Users\CSY\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\CSY\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.


processing valid...


C:\Users\CSY\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.


processing test...


C:\Users\CSY\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.


In [38]:
#KERAS MODEL DEFINITION
from keras.layers import Dense, Dropout, Embedding
from keras.layers import Flatten, Input, SpatialDropout1D, Concatenate
from keras.models import Model
from keras.optimizers import Adam 
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

def get_callbacks(filepath, patience=5):
    es = EarlyStopping('val_loss', patience=patience, restore_best_weights=True)
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]

def get_model():
    embed_dim = 50
    dropout_rate = 0.9
    emb_dropout_rate = 0.9
   
    input_text = Input(shape=[maxlen], name="stem_input")
    input_num = Input(shape=[X_train["num_input"].shape[1]], name="num_input")
    input_svd = Input(shape=[X_train["svd_vect"].shape[1]], name="svd_vect")
    
    emb_lstm = SpatialDropout1D(emb_dropout_rate) (Embedding(n_stem_seq, embed_dim,input_length = maxlen) (input_text))
    concatenate = Concatenate()([(Flatten() (emb_lstm)), input_num, input_svd])
    dense = Dropout(dropout_rate) (Dense(256) (concatenate))
    
    output = Dense(3, activation="softmax")(dense)

    model = Model([input_text, input_num, input_svd], output)

    optimizer = Adam(lr=0.0008, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer)
    return model

model = get_model()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
stem_input (InputLayer)         (None, 60)           0                                            
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, 60, 50)       757250      stem_input[0][0]                 
__________________________________________________________________________________________________
spatial_dropout1d_13 (SpatialDr (None, 60, 50)       0           embedding_13[0][0]               
__________________________________________________________________________________________________
flatten_13 (Flatten)            (None, 3000)         0           spatial_dropout1d_13[0][0]       
__________________________________________________________________________________________________
num_input 

In [42]:
#TRAIN KERAS MODEL
file_path = ".model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=10)

model = get_model()
model.fit(X_train, y_train, epochs=150, validation_data=[X_valid, y_valid], batch_size=512, callbacks = callbacks)

Train on 13705 samples, validate on 5874 samples
Epoch 1/150
13705/13705 [==============================] - 4s 295us/step - loss: 1.1445 - val_loss: 1.0204
Epoch 2/150
13705/13705 [==============================] - 2s 182us/step - loss: 1.0753 - val_loss: 1.0014ETA: 1s -
Epoch 3/150
13705/13705 [==============================] - 3s 183us/step - loss: 1.0413 - val_loss: 0.9885
Epoch 4/150
13705/13705 [==============================] - 2s 175us/step - loss: 1.0215 - val_loss: 0.9734 ETA: 1s -  - ETA: 0s - loss: 
Epoch 5/150
13705/13705 [==============================] - 2s 176us/step - loss: 1.0030 - val_loss: 0.9532
Epoch 6/150
13705/13705 [==============================] - 2s 181us/step - loss: 0.9800 - val_loss: 0.9279
Epoch 7/150
13705/13705 [==============================] - 3s 206us/step - loss: 0.9484 - val_loss: 0.8929
Epoch 8/150
13705/13705 [==============================] - 3s 191us/step - loss: 0.9110 - val_loss: 0.8476
Epoch 9/150
13705/13705 [==============================]

In [43]:
#MODEL EVALUATION
from sklearn.metrics import log_loss

model = get_model()
model.load_weights(file_path)

preds_train = model.predict(X_train)
preds_valid = model.predict(X_valid)

print(log_loss(y_train, preds_train))
print(log_loss(y_valid, preds_valid))

0.12529461447243231
0.40399417783453656


In [29]:
#PREDICTION
preds = pd.DataFrame(model.predict(X_test), columns=target_vars)
submission = pd.concat([test["id"],preds], axis=1)
submission.to_csv("keras.csv", index=False)
submission.head()

,id,EAP,HPL,MWS
0,id02310,0.025837,0.002814,0.971349
1,id24541,0.976705,0.019966,0.003329
2,id00134,0.013211,0.985711,0.001078
3,id27757,0.678175,0.312398,0.009427
4,id04081,0.974057,0.008208,0.017735
